In [2]:
%matplotlib ipympl
from models import *
# from models import find_index_of_nearest
from impedance.preprocessing import readCSV, readBioLogic
from impedance.visualization import plot_nyquist,plot_bode
from impedance.models.circuits import CustomCircuit
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
# from arbitau import give_z_for_arbitary_tau, tau_plotter
from scipy.integrate import solve_bvp
# import mps


In [3]:
@element(num_params=4,units=["","","","",""],overwrite=True)
def arbitRCPEone(p,f):
    Q, alpha, a, b = p[0], p[1], p[2], p[3]
    
    def R(x):
        # return a/(1+b*x)
        return 100*(a*np.tanh(b**2-a*x)+np.tanh(a-b**2*x))
    
    def fun(x,y,lmdcos,lmdsin):
        return np.vstack((y[2]*R(x),y[3]*R(x),lmdcos*y[0]-lmdsin*y[1],lmdcos*y[1]+lmdsin*y[0]))

    def bc(ya,yb):
        return np.array([ya[0]-1,yb[2],yb[3],ya[1]])

    x=np.linspace(0,1,1000)
    y=np.zeros((4,x.size))
 
    Z_list=np.zeros((np.size(np.array(f))),dtype=complex)
    ii=0
    for freq in np.array(f):
        w=2*np.pi*freq
        lmdcos=w**alpha*Q*np.cos(alpha*np.pi/2)
        lmdsin=w**alpha*Q*np.sin(alpha*np.pi/2)
        sol1 = solve_bvp(lambda x,y: fun(x,y,lmdcos=lmdcos,lmdsin=lmdsin), bc, x, y)
        y3=sol1.sol(x)[2]
        y4=sol1.sol(x)[3]
        Z=2/(-(y3[0]+y4[0]*1j))
        Z_list[ii]=Z
        ii=ii+1
    return Z_list


In [4]:
@element(num_params=4,units=["","","","",""],overwrite=True)
def arbitRCPEtwo(p,f):
    Q, alpha, a, b = p[0], p[1], p[2], p[3]
    
    def R(x):
        return a/(1+b*x)
        # return 100*(a*np.tanh(b**2-a*x)+np.tanh(a-b**2*x))
    
    def fun(x,y,lmdcos,lmdsin):
        return np.vstack((y[2]*R(x),y[3]*R(x),lmdcos*y[0]-lmdsin*y[1],lmdcos*y[1]+lmdsin*y[0]))

    def bc(ya,yb):
        return np.array([ya[0]-1,yb[2],yb[3],ya[1]])

    x=np.linspace(0,1,1000)
    y=np.zeros((4,x.size))
 
    Z_list=np.zeros((np.size(np.array(f))),dtype=complex)
    ii=0
    for freq in np.array(f):
        w=2*np.pi*freq
        lmdcos=w**alpha*Q*np.cos(alpha*np.pi/2)
        lmdsin=w**alpha*Q*np.sin(alpha*np.pi/2)
        sol1 = solve_bvp(lambda x,y: fun(x,y,lmdcos=lmdcos,lmdsin=lmdsin), bc, x, y)
        y3=sol1.sol(x)[2]
        y4=sol1.sol(x)[3]
        Z=2/(-(y3[0]+y4[0]*1j))
        Z_list[ii]=Z
        ii=ii+1
    return Z_list

In [5]:
freq_range=np.logspace(-2,3,100)

In [8]:
#common Params
Q=6e-4
alpha=0.9

In [6]:
#prof 1

aone=4
bone=4
paramone=[Q,alpha,aone,bone]

cone=CustomCircuit(initial_guess=paramone,circuit="arbitRCPEone")
zone=cone.predict(frequencies=freq_range)

c:\Users\Gaming\AppData\Local\Programs\Python\Python311\Lib\site-packages\impedance\models\circuits\circuits.py:145: UserWarning: Simulating circuit based on initial parameters
  warnings.warn("Simulating circuit based on initial parameters")


In [9]:
#prof 2

atwo=1000
btwo=1
paramtwo=[Q,alpha,atwo,btwo]

ctwo=CustomCircuit(initial_guess=paramtwo,circuit="arbitRCPEtwo")
ztwo=ctwo.predict(frequencies=freq_range)

In [20]:
R1=1000
R2=500
Q=1e-3
alpha=0.95
delta=0.2063

paramstep=[R1,R1*Q,R2*Q]
paramlin=[R1,R1*Q,R2*Q]
paramilin=[R1,R1*Q,R2*Q]

cstep=CustomCircuit(initial_guess=paramstep,circuit="TLMtwo",constants={"TLMtwo_3":0.2063, "TLMtwo_4":0.9})
clin=CustomCircuit(initial_guess=paramlin,circuit="TLMlinnew",constants={"TLMlinnew_3":0.9})
cilin=CustomCircuit(initial_guess=paramilin,circuit="TLMilinnew",constants={"TLMilinnew_3":0.9})


In [ ]:
#step case
cstep.fit(impedance=zone,frequencies=freq_range)
step_R1=cstep.parameters_[0]
step_Q=cstep.parameters_[1]/cstep.parameters_[0]
step_R2=cstep.parameters_[2]/step_Q
# step_alpha=cstep.parameters_[3]
# cstep.parameters_[0]

# print(cstep)

In [22]:
#lin case
clin.fit(impedance=zone,frequencies=freq_range)
lin_R1=clin.parameters_[0]
lin_Q=clin.parameters_[1]/cstep.parameters_[0]
lin_R2=clin.parameters_[2]/step_Q

In [25]:
#ilin case
mp.dps=200
cilin.fit(impedance=zone,frequencies=freq_range)
ilin_R1=cilin.parameters_[0]
ilin_Q=cilin.parameters_[1]/cstep.parameters_[0]
ilin_R2=cilin.parameters_[2]/step_Q

ZeroDivisionError: 